In [1]:
%cd /home/q123/Desktop/explo/

import gym
import numpy as np
import torch
from src.environment import EnvironmentObjective



class BasicNumGrad:
    
    def __init__(self, env, policy,policy_params,
                 params_per_step,episodes_per_param,n_workers):
        
        self.env_obj = EnvironmentObjective(env, policy)
        self.policy_params = policy_params
        self.optimizer = torch.optim.Adam([self.policy_params])
        
        
    def approx_grad(self, sigma=1e-1, roll_per_parms=10, param_per_step=10):
        
        policy_params = self.policy_params
        all_e, all_f = [], []
        
        for k in range(param_per_step):
            
            all_e.append(torch.randn_like(policy_params) * sigma)
            self.env_obj.mlp.set_weights(policy_params + all_e[-1])
            all_f.append(self.env_obj.run_many(None, roll_per_parms)[0].item())

            # mirror perturbation
            all_e.append(-all_e[-1])
            self.env_obj.mlp.set_weights(policy_params + all_e[-1])
            all_f.append(self.env_obj.run_many(None, roll_per_parms)[0].item())

        nb_evals = len(all_f)

        # weights = np.asarray(all_f)
        # weights = (weights - weights.mean())  # also seems to do ok
        weights = np.argsort(all_f).argsort() / (nb_evals - 1) - .5

        return sum([w * e for w, e in zip(weights, all_e)]) / sigma / nb_evals, mu, all_f

    def optimize(self, step_size=1e-3, nb_steps=100, **approx_grad_kwargs):
        for k in range(nb_steps):
            grad, old_par, evals = self.approx_grad(**approx_grad_kwargs)
            print(f'iteration {k}, mean {sum(evals)/len(evals)} max {max(evals)}')
            adam_grad = self.adamize(grad, k + 1)
            self.env_obj.mlp.set_weights(old_par + step_size * adam_grad)


# if __name__ == '__main__':
#     torch.set_num_threads(1)
#     # env = gym.make('Pendulum-v1')
#     env = gym.make('Walker2DBulletEnv-v0')
#     mlp = MLPSequential([env.observation_space.shape[0], 64, 64, env.action_space.shape[0]])

#     # num grad
#     optimizer = BasicNumGrad(env, mlp)
#     optimizer.optimize(step_size=1e-3, nb_steps=500, sigma=2e-2, roll_per_parms=10, param_per_step=10)


/home/q123/Desktop/explo


/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np 
import multiprocessing as mp
import torch
from copy import deepcopy


class ESOptimizer(object):
    
    
    def __init__(self,env,params_init,sigma,episodes_per_param):
        
        pass

    def run_noisy_params(env,params,
                        seed,sigma,episodes_per_param):
        
        """ Generate and run a pair of symmetric noisy parameters

        Returns:
            weighted_noise: J(theta+eps)*eps
        """
        
        ## Generate noisy parameters
        ## Noise is different for each seed
        torch.manual_seed(seed)
        eps = torch.randn_like(params) * sigma
        
        params1 = params + eps
        params2 = params - eps ## symmetric noise

        tmp_env = deepcopy(env)
        rewards1,states1 = tmp_env.run_many(params1,episodes_per_param)
        rewards2,states2 = tmp_env.run_many(params2,episodes_per_param)
        
        weighted_noise = (1/sigma)*(rewards1*eps - rewards2*eps)*0.5 ## 
        
        return weighted_noise



    def compute_gradient(env,params,sigma,
                    params_per_step,episodes_per_param,n_workers=None):

        args = [(env,params,i,1,episodes_per_param) for i in range(params_per_step)]
        
        # Step 1: Init multiprocessing.Pool()

        if n_workers is None : n_workers = mp.cpu_count()
        pool = mp.Pool(n_workers)

        # Step 2:  Run processes (we might need to use mapreduce to avoid big memory usage)
        weighted_noises = pool.starmap(run_noisy_params,args) ## list of [(reward,eps)]

        # Step 3: Wait for workers to run then close pool
        pool.close()
        pool.join()

        gradient  = torch.vstack(weighted_noises).mean(dim=0)
        
        return gradient
    
    
    def step(self):
        

In [3]:
%cd /home/q123/Desktop/explo/
from src.helpers import setup_experiment
from src.config import get_configs
import torch

env_name = "Swimmer-v4"
kernel_name = "rbf"

env_config,likelihood_config,kernel_config,optimizer_config,trainer_config = get_configs(env_name,kernel_name)
_,env = setup_experiment(env_config,kernel_config,likelihood_config,additional_layers=[])


# grad = compute_gradient(env,torch.zeros(18),sigma=1e-2,
#                 params_per_step=50,episodes_per_param=1,n_workers=8)

/home/q123/Desktop/explo
MathLog.src.helpers : WARNING : MLP dimensions : [8, 2]


/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:69: UserWarning: WARN: Agent's minimum action space value is -infinity. This is probably too low.
  logger.warn(
/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:73: UserWarning: WARN: Agent's maximum action space value is infinity. This is probably too high
  logger.warn(
/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


Using ard_num_dims = 18


In [21]:
grad1 = compute_gradient(env,torch.zeros(18),sigma=1e-6,
                params_per_step=200,episodes_per_param=5,n_workers=8)

grad2 = compute_gradient(env,torch.zeros(18),sigma=1e-6,
                params_per_step=400,episodes_per_param=5,n_workers=8)

print(grad1)
print(grad2)

tensor([ 0.0076,  0.0013, -0.0323, -0.0074, -0.0014, -0.0015, -0.0093,  0.0106,
        -0.0187,  0.0316, -0.0071, -0.0038,  0.0127, -0.0035,  0.0080, -0.0112,
         0.0071, -0.0030])
tensor([ 6.6228e-03, -8.3212e-04, -2.4226e-02, -7.8172e-03,  5.2993e-03,
        -5.2986e-03, -9.9881e-05,  1.2232e-02, -2.4621e-02,  3.5464e-02,
        -9.2076e-03, -3.2949e-03,  9.3940e-03, -4.2922e-03,  1.3703e-02,
        -5.1019e-03,  3.0720e-03, -2.1394e-03])


In [23]:
(grad2/grad1).mean()

tensor(0.7038)

In [6]:
print("hi")

hi
